# Data imbalance analysis
## Comparing the impact of dataset imbalance on classification performance metrics
In this notebook, we are going to explore how data imbalance affect classification scores on four different tasks, which have been chosen to exemplify typical use cases of ML analysis in neuroscience. To keep it simple, we will focus on binary classification problems (0 vs 1).

The four tasks are :
1. Synthetic data
2. EEG alpha oscillations (resting-state Eyes-Closed vs Eyes-Open)
3. MEG alpha oscillations (auditory vs visual stimulation)
4. MEG alpha oscillations (faces vs scrambled) 

In these tasks, we will observe the effect of data imbalance on 4 different performance metrics :   
1. Decoding Accuracy (Acc)  
2. Area Under the Curve (AUC)
3. F1
4. Balanced Accuracy (BAcc)

Finally, a few parameters of the classification pipeline must be kept in mind as they can also differentially impact performance on imbalanced data. Namely :
- Dataset size
- Classifier type
    - Support Vector Machine (SVM)
    - Linear Discriminant Analysis (LDA)
    - Logistic Regression (LR)
    - Random Forest (RF)
- Cross-validation scheme
    - K-Fold, k=5
    - Stratified K-Fold
    - Group K-Fold

## Imports
First, we start by importing functions from the provided toolbox as well as some useful plotting functions.

In [9]:
from imbalance.pipeline import Pipeline
from imbalance.viz import metric_balance, data_distribution
from imbalance.data import eegbci, gaussian_binary
from joblib import Parallel, delayed
from sklearn.model_selection import (
    KFold,
    StratifiedKFold,
    StratifiedGroupKFold,
)
import os
import numpy as np
import matplotlib.pyplot as plt
import string
from copy import deepcopy
import warnings
warnings.simplefilter("ignore", RuntimeWarning)

## Task 1 : Synthetic data
For that first classification task, we will generate data from two gaussian distributions, with means of 0 and 2.

In [2]:
pls = []
#for distance in [0,1,3]:
    
def run(distance):
    # generate random data
    x, y, groups = gaussian_binary(n_samples_per_class=500, mean_distance=distance)
    # run the pipeline
    pl = Pipeline(
        x,
        y,
        groups,
        dataset_balance=np.linspace(0.1, 0.9, 25),
        classifiers=["lr", "lda", "svm"],# "rf"],
        metrics=["roc_auc", "accuracy", "f1", "balanced_accuracy"],
        n_permutations=100,
        n_init=10,
    )
    pl.evaluate()
    return deepcopy(pl)



pls = Parallel(n_jobs=3)(delayed(run)(dist) for dist in [0, 1, 3])

fitting classifiers: 100%|██████████| 750/750 [06:13<00:00,  2.01it/s, size=1, balance=0.9, classifier=SVC]                          

In [ ]:
# generate random data
x, y, groups = gaussian_binary(n_samples_per_class=1500, n_groups=5, mean_distance=1)
# run the pipeline
pl_nsamples = Pipeline(
    x,
    y,
    groups,
    dataset_balance=np.linspace(0.1, 0.9, 25),
    classifiers=["svm"],# "rf"],
    metrics=["roc_auc", "accuracy", "f1", "balanced_accuracy"],
    n_permutations=0,
    n_init=1,
    dataset_size=(0.1, 0.33, 1)
)
pl_nsamples.evaluate()

fitting classifiers:   0%|                              | 0/75 [00:00<?, ?it/s, size=0.1, balance=0.1, classifier=SVC]

In [ ]:

pls_crossvals = []
cvs = ["KFold", "Stratified", "Stratified Group"]

# generate random data
x, y, groups = gaussian_binary(n_samples_per_class=500, n_groups=5, mean_distance=1)
# run the pipeline
for cross_val in [KFold(n_splits=5), StratifiedKFold(n_splits=5), StratifiedGroupKFold(n_splits=5)]:
    pl = Pipeline(
        x,
        y,
        groups,
        dataset_balance=np.linspace(0.1, 0.9, 25),
        classifiers=["lr", "lda", "svm"],# "rf"],
        metrics=["roc_auc", "accuracy", "f1", "balanced_accuracy"],
        n_permutations=100,
        n_init=10,
        cross_validation=cross_val,
    )
    pl.evaluate()
    pls_crossvals.append(deepcopy(pl))
    





fitting classifiers:   0%|        | 1/750 [00:02<33:25,  2.68s/it, size=1, balance=0.1, classifier=LogisticRegression]
fitting classifiers:   0%| | 1/750 [00:02<33:25,  2.68s/it, size=1, balance=0.1, classifier=LinearDiscriminantAnalysis
fitting classifiers:   0%| | 2/750 [00:04<23:26,  1.88s/it, size=1, balance=0.1, classifier=LinearDiscriminantAnalysis
fitting classifiers:   1%|      | 4/750 [00:07<20:53,  1.68s/it, size=1, balance=0.133, classifier=LogisticRegression]
fitting classifiers:   1%| | 4/750 [00:07<20:53,  1.68s/it, size=1, balance=0.133, classifier=LinearDiscriminantAnalys
fitting classifiers:   1%| | 5/750 [00:08<20:01,  1.61s/it, size=1, balance=0.133, classifier=LinearDiscriminantAnalys
fitting classifiers:   1%|      | 7/750 [00:11<19:49,  1.60s/it, size=1, balance=0.167, classifier=LogisticRegression]
fitting classifiers:   1%| | 7/750 [00:11<19:49,  1.60s/it, size=1, balance=0.167, classifier=LinearDiscriminantAnalys
fitting classifiers:   1%| | 8/750 [00:13<18:45

fitting classifiers:   5%|▏    | 34/750 [01:07<31:12,  2.62s/it, size=1, balance=0.467, classifier=LogisticRegression]
fitting classifiers:   5%| | 34/750 [01:07<31:12,  2.62s/it, size=1, balance=0.467, classifier=LinearDiscriminantAnaly
fitting classifiers:   5%| | 35/750 [01:09<26:38,  2.24s/it, size=1, balance=0.467, classifier=LinearDiscriminantAnaly
fitting classifiers:   5%|▎      | 37/750 [01:16<32:24,  2.73s/it, size=1, balance=0.5, classifier=LogisticRegression]
fitting classifiers:   5%| | 37/750 [01:16<32:24,  2.73s/it, size=1, balance=0.5, classifier=LinearDiscriminantAnalysi
fitting classifiers:   5%| | 38/750 [01:17<27:35,  2.33s/it, size=1, balance=0.5, classifier=LinearDiscriminantAnalysi
fitting classifiers:   5%|▎    | 40/750 [01:25<34:01,  2.87s/it, size=1, balance=0.533, classifier=LogisticRegression]
fitting classifiers:   5%| | 40/750 [01:25<34:01,  2.87s/it, size=1, balance=0.533, classifier=LinearDiscriminantAnaly
fitting classifiers:   5%| | 41/750 [01:26<29:13

fitting classifiers:   9%|▍    | 67/750 [02:33<21:22,  1.88s/it, size=1, balance=0.833, classifier=LogisticRegression]
fitting classifiers:   9%| | 67/750 [02:33<21:22,  1.88s/it, size=1, balance=0.833, classifier=LinearDiscriminantAnaly
fitting classifiers:   9%| | 68/750 [02:35<21:27,  1.89s/it, size=1, balance=0.833, classifier=LinearDiscriminantAnaly
fitting classifiers:   9%|▍    | 70/750 [02:40<23:20,  2.06s/it, size=1, balance=0.867, classifier=LogisticRegression]
fitting classifiers:   9%| | 70/750 [02:40<23:20,  2.06s/it, size=1, balance=0.867, classifier=LinearDiscriminantAnaly

In [ ]:
# visualize the result
fig, axes = plt.subplots(6, 3, figsize=(20, 30))
figtitle = "Synthetic data"
#fig.suptitle(figtitle, fontsize=25)
classifiers=["lr", "lda", "svm", "rf"]

show_leg_distrib=True
show_leg_metric=True
for ax_idx,ax in enumerate(axes.flat):
    ax.text(-0.1, 1.05, string.ascii_uppercase[ax_idx], transform=ax.transAxes, 
            size=20, weight='bold')
    if ax_idx in [0,1,2]:
        data_distribution(pls[ax_idx], ax=ax, show=False, show_leg=show_leg_distrib)
        show_leg_distrib=False
    elif ax_idx < 15:
        metric_balance(pls[ax_idx%3], ax=ax, show=False, classifier=classifiers[(ax_idx-3)//3], show_leg=show_leg_metric)
        show_leg_metric = False
    elif ax_idx == 15:
        metric_balance(pls_nsamples[0], ax=ax, show=False, classifier="svm", show_leg=True)

## Task 2 : EEG analysis

In [ ]:
#pipeline_path="../imbalance/data/eeg.pickle"
features_path ="../imbalance/data/eeg_features.npy"

# load or generate dataset
if not os.path.isfile(features_path):
    x, y, groups = eegbci('../imbalance/data',roi=lambda x: x[0] in ['P','O'])
    np.save(features_path,dict(x=x, y=y, groups=groups))
else:
    features = np.load(features_path,allow_pickle=True).item()
    x, y, groups = features["x"] , features["y"] , features["groups"]

In [ ]:
pl = Pipeline(
    x,
    y,
    groups,
    dataset_balance=np.linspace(0.1, 0.9, 25),
    classifiers=["lda","svm","lr"],
    metrics=[ "roc_auc","accuracy", "f1", "balanced_accuracy"],
)
# fit and evaluate classifiers on dataset configurations
pl.evaluate()

In [ ]:
# visualize the result
fig, axes = plt.subplots(2, 3, figsize=(20, 10))
figtitle = "EEG data"
#fig.suptitle(figtitle, fontsize=25)
classifiers=["lr", "lda", "svm", "rf"]

for ax_idx,ax in enumerate(axes.flat):
    ax.text(-0.1, 1.05, string.ascii_uppercase[ax_idx], transform=ax.transAxes, 
            size=20, weight='bold')
    if ax_idx == 0:
        data_distribution(pl, ax=ax, show=False)
    elif ax_idx < 6:
        metric_balance(pl, ax=ax, show=False, classifier=classifiers[ax_idx-1])

## Task 3 : MEG 1

## Task 4 : MEG 2